# 2. TensorBoard: Hidden Vector Visualization / Embedding

이번 튜토리얼에서는 CNN을 학습시키고 softmax layer 직전의 fully connected layer에서의 hidden vector를 embedding visulization을 해보도록 하겠습니다.

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector
import tensorflow.contrib.slim as slim
import tensorflow as tf
import numpy as np

### Load MNIST dataset
이전 튜토리얼과 마찬가지로 MNIST 데이터셋을 불러옵니다. 이번에는 CNN이기 때문에 reshape=False로 했습니다.

In [2]:
mnist = input_data.read_data_sets(train_dir='mnist', reshape=False)
x_train = mnist.train.images
y_train = mnist.train.labels
x_test = mnist.test.images
y_test = mnist.test.labels

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


### Define the Model
아래 코드에서는 TensorFlow 공식 high-level API인 slim을 사용해서 CNN 모델을 정의합니다. 그 동안 tutorial에서 배웠던 것을 복습할 겸 pass 부분을 구현하시면 됩니다.

In [3]:
class Convnet(object):
    def __init__(self, learning_rate=0.001):
        self.images = tf.placeholder(tf.float32, shape=[None, 28, 28, 1], name='input_images')
        self.targets = tf.placeholder(tf.int64, shape=[None], name='target_indices')
        self.learning_rate = learning_rate
    
        # build model
        with slim.arg_scope([slim.conv2d], padding='SAME', activation_fn=tf.nn.relu, stride=2,
                            weights_initializer=tf.contrib.layers.xavier_initializer()):
            
            conv1 = slim.conv2d(self.images, 16, [3, 3], scope='conv1')                              # (batch_size, 14, 14, 16)  
            conv2 = slim.conv2d(conv1, 32, [3, 3], scope='conv2')                                    # (batch_size, 7, 7, 32)
            conv3 = slim.conv2d(conv2, 64, [3, 3], scope='conv3')                                    # (batch_size, 4, 4, 64)
            conv4 = slim.conv2d(conv3, 128, [4, 4], padding='VALID', scope='conv4')                  # (batch_size, 1, 1, 128)
            out = slim.flatten(slim.conv2d(conv4, 10, [1, 1], activation_fn=None, scope='out'))      # (batch_size, 10)
            
            # features we extract
            self.fc = slim.flatten(conv4)  # (batch_size, 128)
            
            # TODO: 아래 부분을 모두 구현하시면 됩니다. (tf.slim을 사용할 필요는 없습니다.)
            pred = pass
            correct_pred = pass
            self.accuracy = pass
            self.loss = pass
            self.train_op = pass

### Construct the Model
모델 그래프를 생성합니다.

In [4]:
model = pass

### Train the Model
CNN 모델을 학습 시킵니다.

In [6]:
num_epoch = 10
batch_size = 100
num_iter_per_epoch = int(x_train.shape[0] / batch_size)

gpu_config = tf.ConfigProto()
gpu_config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=gpu_config) 

log_dir = 'logs/'
summary_writer = tf.summary.FileWriter(logdir=log_dir, graph=tf.get_default_graph())
tf.global_variables_initializer().run()
for e in range(num_epoch):
    avg_loss = 0.0
    avg_acc = 0.0
    for i in range(num_iter_per_epoch):
        # TODO: batch data와 feed_dict를 설정하고 session을 실행시켜 학습을 시키도록 합니다.
        x_batch = pass
        y_batch = pass
        feed_dict = pass
        _, l, acc = pass
        
        avg_loss += l / num_iter_per_epoch
        avg_acc += acc / num_iter_per_epoch
    
    print('Epoch [%d/%d], Loss: %.4f, Accuracy: %.4f' %(e+1, num_epoch, avg_loss, avg_acc))

Epoch [1/10], Loss: 0.3206, Accuracy: 0.9062
Epoch [2/10], Loss: 0.0919, Accuracy: 0.9720
Epoch [3/10], Loss: 0.0602, Accuracy: 0.9818
Epoch [4/10], Loss: 0.0443, Accuracy: 0.9862
Epoch [5/10], Loss: 0.0331, Accuracy: 0.9898
Epoch [6/10], Loss: 0.0263, Accuracy: 0.9918
Epoch [7/10], Loss: 0.0208, Accuracy: 0.9936
Epoch [8/10], Loss: 0.0191, Accuracy: 0.9936
Epoch [9/10], Loss: 0.0171, Accuracy: 0.9941
Epoch [10/10], Loss: 0.0129, Accuracy: 0.9960


### Extract the Hidden Vectors
MNIST 테스트 이미지를 학습된 CNN에 forward propagation시켜 128x1짜리 hidden vector를 추출합니다.

In [8]:
num_iter = int(x_test.shape[0] / batch_size)
features = np.zeros((x_test.shape[0], 128))

for i in range(num_iter):
    # TODO: batch data를 설정하고 hidden vectors를 추출합니다.
    x_batch = pass
    y_batch = pass
    features[i*batch_size:(i+1)*batch_size] = pass

### Embedding and Visualizing the Hidden Vectors
아래부터는 이전 exercise와 동일하므로 새로 구현할 필요없이 실행만 하시면 됩니다. 

In [9]:
embedding_var = tf.Variable(features, name='embedding')

In [10]:
# make log directory if not exists
log_dir = 'logs/'
if tf.gfile.Exists(log_dir):
    tf.gfile.DeleteRecursively(log_dir)
tf.gfile.MakeDirs(log_dir)

In [11]:
tf.global_variables_initializer().run()
saver = tf.train.Saver()
saver.save(sess, save_path=log_dir+'model.ckpt', global_step=None)

'logs/model.ckpt'

In [12]:
# add embedding data
summary_writer = tf.summary.FileWriter(logdir=log_dir, graph=tf.get_default_graph())
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name

In [13]:
# add label data
metadata_path = 'logs/metadata.tsv'
with open(metadata_path, 'w') as f:
    for i, label in enumerate(mnist.test.labels):
        f.write('{}\n'.format(label))
embedding.metadata_path = metadata_path

In [14]:
# add sprite image
embedding.sprite.image_path = 'mnist/mnist_10k_sprite.png'
embedding.sprite.single_image_dim.extend([28, 28])

In [15]:
# visualize embedding projector
projector.visualize_embeddings(summary_writer, config)

이전 exercise와 마찬가지로, 새로운 linux terminal을 키시고 아래처럼 명령어를 입력합니다. 그 다음 웹 브라우저에 `163.152.20.66:6006`를 입력하여 tensorboard를 실행하면 embedding 결과를 확인할 수 있습니다.

```bash
source anaconda2/bin/activate ~/anaconda2
cd davian-tensorflow/notebooks/week5/
tensorboard --logdir=logs --port=6005
```

### Embedding Result
아래는 PCA를 사용하여 mnist 이미지의 hidden vector를 embedding 했을 때의 모습입니다. 

![alt text](jpg/pca_cnn.jpg)